In [1]:
import torch
from transformers import SegformerForImageClassification

# Load Alzheimer's model
alzheimers_model = SegformerForImageClassification.from_pretrained('nvidia/mit-b1')
alzheimers_model.classifier = torch.nn.Linear(alzheimers_model.classifier.in_features, 4)  # 4 classes
alzheimers_model.load_state_dict(torch.load('alzheimers_model.pth', map_location=torch.device('cpu')))
alzheimers_model.eval()

# Load Brain Tumor model
brain_tumor_model = SegformerForImageClassification.from_pretrained('nvidia/mit-b1')
brain_tumor_model.classifier = torch.nn.Linear(brain_tumor_model.classifier.in_features, 4)  # 4 classes
brain_tumor_model.load_state_dict(torch.load('brain_tumor_model.pth', map_location=torch.device('cpu')))
brain_tumor_model.eval()

SegformerForImageClassification(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)


In [2]:
from torchvision import transforms
from PIL import Image

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Prediction function for Alzheimer's
def predict_alzheimers(image):
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = alzheimers_model(image).logits
    _, predicted = torch.max(outputs, 1)
    classes = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
    return classes[predicted.item()]

# Prediction function for Brain Tumor
def predict_brain_tumor(image):
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = brain_tumor_model(image).logits
    _, predicted = torch.max(outputs, 1)
    classes = ['glioma', 'meningioma', 'notumor', 'pituitary']
    return classes[predicted.item()]

In [ ]:
import gradio as gr
def predict(image, model_type):
    if model_type == "Alzheimer's":
        return predict_alzheimers(image)
    elif model_type == "Brain Tumor":
        return predict_brain_tumor(image)

interface = gr.Interface(
    fn=predict,
    inputs=[gr.Image(type="pil"), gr.Dropdown(["Alzheimer's", "Brain Tumor"])],
    outputs=gr.Textbox(),
    title="MRI Scan Classification",
    description="Upload an MRI scan and select the type of classification."
)

interface.launch()